In [2]:
import tiktoken
from openai import OpenAI

tokenizer = tiktoken.get_encoding("cl100k_base")
client = OpenAI()

In [3]:
reducing_example0 = """Q: The median age in the city was 22.1 years. 10.1% of residents were under the age of 18; 56.2% were between the ages of 18 and 24; 16.1% were from 25 to 44; 10.5% were from 45 to 64; and 7% were 65 years of age or older. Which age group is larger: under the age of 18 or 18 and 24?
A: To answer the question “Which age group is larger: under the age of 18 or 18 and 24?”, we need to know: “How many percent were under the age of 18?”, “How many percent were between the ages of 18 and 24?”."""
reducing_example1 = """Q: Old age pensions were raised by 300 francs per month to 1,700 francs for a single person and to 3,700 francs for a couple, while health insurance benefits were made more widely available to unemployed persons and part-time employees. How many francs were the old age pensions for a single person before they were raised?
A: To answer the question “How many francs were the old age pensions for a single person before they were raised?”, we need to know: “How many francs were the old age pensions for a single person?”, “How many francs were old age pensions raised for a single person?”."""
reducing_example2 = """Q: In April 2011, the ECB raised interest rates for the first time since 2008 from 1% to 1.25%, with a further increase to 1.50% in July 2011. However, in 2012-2013 the ECB lowered interest rates to encourage economic growth, reaching the historically low 0.25% in November 2013. Soon after the rates were cut to 0.15%, then on 4 September 2014 the central bank reduced the rates from 0.15% to 0.05%, the lowest rates on record. How many percentage points did interest rates drop between April 2011 and September 2014?
A: To answer the question “How many percentage points did interest rates drop between April 2011 and September 2014?”, we need to know: “What was the interest rate in April 2011?”, “What was the interest rate in September 2014?”."""
reducing_example3 = """Q: Non-nationals make up more than half of the population of Bahrain. According to government statistics dated between 2005-2009 roughly 290,000 Indians, 125,000 Bangladeshis, 45,000 Pakistanis, 45,000 Filipinos, and 8,000 Indonesians. How many Pakistanis and Indonesians are in Bahrain?
A: To answer the question “How many Pakistanis and Indonesians are in Bahrain?”, we need to know: “How many Pakistanis are in Bahrain?”, “How many Indonesians are in Bahrain?”."""
reducing_example4 = """Q: The Census Bureaus 2006-2010 American Community Survey showed that (in 2010 inflation adjustment dollars) median household income was $52,056 and the median family income was $58,942. How many years did the Census Bureaus American Community Survey last?
A: To answer the question “How many years did the Census Bureaus American Community Survey last?”, we need to know: “When did the Census Bureaus American Community Survey start?”, “When did the Census Bureaus American Community Survey end?”."""

reducing_examples = "\n\n".join(
    [
        f"""Example {i}:
```
{example}
```"""
        for i, example in enumerate(
            [
                reducing_example0,
                reducing_example1,
                reducing_example2,
                reducing_example3,
                reducing_example4,
            ]
        )
    ]
)

In [4]:
reducing_system_message = f"""You are an AI assistant tasked with identifying the key pieces of information needed to answer a given question. Your goal is to break down the question into its essential components without actually answering it.

Here are some examples.

{reducing_examples}

(END OF EXAMPLES)
"""

reducing_system_message_tokens = len(tokenizer.encode(reducing_system_message))
print("Tokens:", reducing_system_message_tokens)
print()
print(reducing_system_message)

Tokens: 793

You are a helpful assistant.

Here are some examples.

Example 0:
```
Q: The median age in the city was 22.1 years. 10.1% of residents were under the age of 18; 56.2% were between the ages of 18 and 24; 16.1% were from 25 to 44; 10.5% were from 45 to 64; and 7% were 65 years of age or older. Which age group is larger: under the age of 18 or 18 and 24?
A: To answer the question “Which age group is larger: under the age of 18 or 18 and 24?”, we need to know: “How many percent were under the age of 18?”, “How many percent were between the ages of 18 and 24?”.
```

Example 1:
```
Q: Old age pensions were raised by 300 francs per month to 1,700 francs for a single person and to 3,700 francs for a couple, while health insurance benefits were made more widely available to unemployed persons and part-time employees. How many francs were the old age pensions for a single person before they were raised?
A: To answer the question “How many francs were the old age pensions for a singl

In [5]:
solving_example0 = """The city of Bangkok has a population of 8,280,925. The census showed that it is home to 81,570 Japanese and 55,893 Chinese nationals, as well as 117,071 expatriates from other Asian countries, 48,341 from Europe, 23,418 from the Americas.
Q: How many Chinese nationals are there?
A: “The census showed that it is home to 81,570 Japanese and 55,893 Chinese nationals”. So the answer is 55,893 Chinese nationals.
Q: How many European nationals are there?
A: “48,341 from Europe”. So the answer is 48,341.
Q: How many more Chinese nationals are there than European nationals?
A: We know that there are 55,893 Chinese nationals. We also know that there are 48,341 European nationals. So there are 55,893 - 48,341 = 7552 more Chinese nationals than European nationals. So the answer is 7552."""
solving_example1 = """The gender distribution of the population was 50.2% male and 49.8% female. Of the adult population, 29 people or 14.6% of the population are between 20 and 29 years old. 28 people or 14.1% are 30 to 39, 36 people or 18.2% are 40 to 49, and 31 people or 15.7% are 50 to 59.
Q: What is the percentage of people between 40 to 49?.
A: “36 people or 18.2% are 40 to 49”. So the answer is 18.2
Q: How many percent of people were not 40 to 49?
A: We know that 18.2% are 40 to 49. So 100% - 18.2% = 81.8% are not 40 to 49. So the answer is 81.8%."""
solving_example2 = """During the 1970s the oil shocks (1973 oil crisis and 1979 energy crisis) hit Iceland hard. Inflation rose to 43% in 1974 and 59% in 1980, falling to 15% in 1987 but rising to 30% in 1988. Iceland experienced moderately strong GDP growth (3% on average) from 1995 to 2004. Growth slowed between 2000 and 2002, but the economy expanded by 4.3% in 2003 and grew by 6.2% in 2004. Growth in 2005 exceeded 6%.
Q: When did the oil crisis occur?
A: “1973 oil crisis and 1979 energy crisis”. So the answer is 1973.
Q: When did the energy crisis occur?
A: “1979 energy crisis”. So the answer is 1979.
Q: How many years passed between the oil and energy crises?
A: We know that the oil crisis occurred in 1973. We also know that the energy crisis occurred in 1979. So 1979 - 1973 = 6 years passed between the oil and energy crises. So the answer is 6."""


solving_examples = "\n\n".join(
    [
        f"""Example {i}:
```
{example}
```"""
        for i, example in enumerate(
            [
                solving_example0,
                solving_example1,
                solving_example2,
            ]
        )
    ]
)

In [7]:
solving_system_message = f"""You are an AI assistant designed to answer questions.

Here are some examples.

{solving_examples}

(END OF EXAMPLES)
"""

solving_system_message_tokens = len(tokenizer.encode(solving_system_message))
print("Tokens:", solving_system_message_tokens)
print()
print(solving_system_message)

Tokens: 715

You are an AI assistant designed to answer questions.

Here are some examples.

Example 0:
```
The city of Bangkok has a population of 8,280,925. The census showed that it is home to 81,570 Japanese and 55,893 Chinese nationals, as well as 117,071 expatriates from other Asian countries, 48,341 from Europe, 23,418 from the Americas.
Q: How many Chinese nationals are there?
A: “The census showed that it is home to 81,570 Japanese and 55,893 Chinese nationals”. So the answer is 55,893 Chinese nationals.
Q: How many European nationals are there?
A: “48,341 from Europe”. So the answer is 48,341.
Q: How many more Chinese nationals are there than European nationals?
A: We know that there are 55,893 Chinese nationals. We also know that there are 48,341 European nationals. So there are 55,893 - 48,341 = 7552 more Chinese nationals than European nationals. So the answer is 7552.
```

Example 1:
```
The gender distribution of the population was 50.2% male and 49.8% female. Of the adu